# Consistência dos Dados

Analisando a consistência dos dados.

### Tarefas:

- ~~(X)~~ Entradas duplicadas (em outro notebook)
- ~~(X)~~ Mais nomes do que CPFs/CNPJs
- ~~(X)~~ CNPJs fantasmas
- ( ) Numero de processos
- ( ) Pagamentos negativos
- ( ) Diferença pros dados do TCM

### Outros problemas:

- Portal de Transparência da Prefeitura solicita a seleção de uma unidade.
A informação da unidade não está disponível nos dados fornecidos para download.

## Conclusões


### CPFs/CNPJs (IDs)

Existem 1206 números de identificação (CPF/CNPJ)
com mais de um credor associado. Este número representa
15,41% de todos os IDs (CPF/CNPJ) registrados.

Destes, a maioria dos registros duplicados são
variações na ortografia no mesmo nome do credor.

No entanto, temos também alguns IDs para o mesmo credor.
E diversos credores inscritos no mesmo ID.

Em especial, 177 credores inscritos no CNPJ do 
gabinete do prefeito; 4 IDs inválidos com 253
credores inscritos sob estes IDs.


### Despesas negativas

Existem 16.494 registros de despesas com valores negativos.
Este numero representa 3.85% do total de despesas registradas.

Em geral, as despesas negativas são acompanhadas de despesas positivas
de valor idêntico (neutralizando o saldo) e de alguma despesa positiva
de valor próximo. 

Minha hipótese é que estas despesas negativas
são registradas para neutralizar erros de cadastro anteriores.

Em vários registros, no campo `bem ou serviço prestado`, consta:
ANULAÇÃO PARCIAL DO EMPENHO, ou ADIANTAMENTO DO EMPENHO.

Tem que verificar direitinho o que são

In [12]:
import pandas as pd

In [2]:
pagamentos = pd.read_csv('pagamentos.csv')
pagamentos.info()
print(pagamentos.nunique())
pagamentos.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 427873 entries, 0 to 427872
Data columns (total 15 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   data_publicacao          427873 non-null  object
 1   fase                     427873 non-null  object
 2   credor                   427873 non-null  object
 3   valor                    427873 non-null  object
 4   numero                   427873 non-null  object
 5   cpf_ou_cnpj              427873 non-null  object
 6   data_pagamento           427873 non-null  object
 7   numero_processo          253290 non-null  object
 8   bem_ou_servico_prestado  420978 non-null  object
 9   natureza                 422333 non-null  object
 10  acao                     407378 non-null  object
 11  funcao                   407414 non-null  object
 12  subfuncao                407405 non-null  object
 13  processo_licitatorio     407396 non-null  object
 14  fonte_recurso       

,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
344073,25/01/2018,LIQUIDAÇÃO,POSTOS RENASCER LTDA,"R$ 2.127,39",00511-18,10.603.039/0002-38,25/01/2018,NaN,REF DESPESA COM AQUISICAO DE COMBUSTIVEL. 18/1...,339030020000 - Combustíveis e Lubrificantes,2015 - Manutencao dos serv.tecnicos administra...,04 - ADMINISTRACAO,122 - ADMINISTRACAO GERAL,PREGAO,0000 - RECURSOS ORDINARIOS
178066,10/02/2014,PAGAMENTO,SAMUEL MACEDO DE SENA,"R$ 500,00",11368/14,105.354.205-44,10/02/2014,190/2013/1111,REF. A LOCA¿¿O DE IM¿VEL ONDE FUNCIONA UM PSF....,339039030000 - Loca,2061 - Atencao B¿sica,10 - SAUDE,301 - ATENCAO BASICA,DISPENSA,0014 - TRANSF.REC. SUS
352278,09/03/2018,LIQUIDAÇÃO,R.J.CONS.MEDICO CIRUG. LTDA,"R$ 2.964,73",01527-18,02.247.818/0001-39,09/03/2018,001/2012,REFERENTE À SERVIÇOS MEDICOS PRESTADOS A PACIE...,"339039950000 - Serviço médico-hospitalar, odon...",2056 - Atencao de Media e Alta Complexibilidad...,10 - SAUDE,302 - ASSISTENCIA HOSPITALAR E AMBUL,CONVITE,0014 - TRANSF.REC. SUS
6734,09/03/2010,PAGAMENTO,FREIRE INFORMATICA LTDA,"R$ 27.470,30",01423/10,00.121.056/0001-22,09/03/2010,34/2009/1111,PAGT. REF. LOCACAO DE SISTEMA DE TECNOLOGIA DE...,339039740000 - Outros Serv. de Terc.,2056 - Atencao de Media e Alta Complexibilidad...,10 - SAUDE,302 - ASSISTENCIA HOSPITALAR E AMBUL,PREGAO,0014 - TRANSF.REC. SUS
252730,27/04/2016,PAGAMENTO,COOPERSADE COOP.S.P.ESP.SAUDE,"R$ 28.182,04",20118-16,03.817.692/0001-53,27/04/2016,347/2015,REF DESPESA COM SERVICO DE LOCACAO DE MAO DE O...,339034010000 - Out.Serv.de Terceriza,2194 - Serv.de Prot.e Atend.Esp.a Fam¿lia e In...,08 - ASSISTENCIA SOCIAL,244 - ASSISTENCIA COMUNITARIA,PREGAO,0029 - TRANSF. REC. - FNAS


In [3]:
from scripts.parsers import currency_to_float

pagamentos["valor"] = pagamentos["valor"].apply(currency_to_float)

[nltk_data] Downloading package stopwords to /home/guiga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Entradas duplicadas

## CPFs e CNPJs (IDs)

Existem 7828 IDs (CPF ou CNPJ) únicos, porém 8831 nomes.
Mil a mais. Por que temos mais nomes que IDs?

In [4]:
ids_df = pagamentos[["credor", "cpf_ou_cnpj"]]
ids_group = ids_df.groupby("cpf_ou_cnpj").agg("nunique").sort_values("credor", ascending=False)
ids_group = ids_group[ids_group["credor"] >  1]
num_ids_total = pagamentos["cpf_ou_cnpj"].nunique()

print(f"Numero total de IDs: {num_ids_total}")
print(f"Numero IDs com mais de um credor: {len(ids_group)}\
    ({round(len(ids_group) *100 / num_ids_total,2)}%)")
ids_group.head(15)

Numero total de IDs: 7828
Numero IDs com mais de um credor: 1206    (15.41%)


,credor
cpf_ou_cnpj,
14.043.574/0001-51,177
01.404.357/0001-51,160
000.000.000-00,62
08.576.590/0001-07,23
00.857.659/0001-07,20
07.520.940/0001-50,19
02.983.473/0001-81,13
05.824.468/0001-14,11
04.063.715/0001-36,11


Da posição 10 em diante, os "múltiplos credores"
são apenas variações da escrita do nome do credor.

Na posição 9, temos os seguintes valores pro ID:

    'IPFS- PENSIONISTAS CAMARA',
    'INST.DE PREV.DE F.SANTANA',
    'IPFS/FOLHA APOSENTADOS 1992',
    'IPFS-APOSENTADOS CAMARA',
    'INST. DE PREV.DE F. SANTANA',
    'ADALGISA RAMOS DE S SERAFIM',
    'ORALE', 
    'PREV.SERV.MUN.FEIRA DE SANTANA-IPFS',
    'INST.DE PREVIDENCIA F.SANTANA',
    'PREV.SERV. MUN. FEIRA DE SANTANA'

Dez valores diferentes pra um mesmo ID.
Posições anteriores a 9 todas possuem múltiplos
nomes associados a um mesmo ID.

As posições 10 e 11 contém dois CNPJs pro ITAU

In [5]:
pos = 3
for name in ids_df[ids_df["cpf_ou_cnpj"] == ids_group.index[pos]]["credor"].unique():
    print(name)

FOLHA DE PGTO CEO
FOLHA DE PAGAMENTO CEREST
FOLHA DE PAGAMENTO SMS
FOLHA PAGAMENTO UBS
FOLHA PAG. FARMACIA POPULAR
FOLHA PAGAMENTO PACS-CONTRATO
FOLHA PAGAMENTO DST/AIDS
FOLHA PAG. ENDEMIAS ESTATUTARI
FOLHA PAG. ENDEMIAS-CONTRATO
FOLHA DE PAG. VIG. EM SAUDE
PESSOAL CIVIL
FOLHA PAGAMENTO PACS ESTATUTAR
FOLHA DE PAGAMENTO SAMU
FOLHA PAG. MEDIA COMPLEXIDADE
FOLHA DE PAGAMENTO CAPS
I.N.S.S. S/PROCESSOS
ALAIDE LIMA DE FREITAS
FOLHA DE PAGAMENTO ESTAGIARIO
CAUCAO
FOLHA PAG. PESSOAL CIVIL
PREV.SERV. MUN. FEIRA DE SANTANA
DESC. JUDICIAL(PENSÃO ALIMENTICIA
I.S.S


### IDs utilizados pra múltiplos nomes

Os IDs utilizados por múltiplos nomes são todos CNPJs,
com exceção do CPF 000.000.000-00, inválido.

A quem pertencem os CNPJs utilizados para múltiplos credores?

Conferi através do
[site da Receita](servicos.receita.fazenda.gov.br/Servicos/cnpjreva/Cnpjreva_Solicitacao.asp).

Segue a lista abaixo:

0) 14.043.574/0001-51 -> Dono: FEIRA DE SANTANA PREFEITURA GABINETE DO PREFEITO;

Também utilizado para 176 outros nomes de credores.
Entre eles:

    PESSOAL CIVIL
    BANCO DO BRASIL S/A CENTRO
    INSS-INSTIT.NAC.SEGURID.SOCIAL
    BANCO DO BRASIL S/A.
    EMPRESTIMO CAIXA
    EMP BB
    I.N.S.S. - FOLHA
    BANCO REAL
    EMPRESTIMO B B
    VIDA AZUL
    I.N.S.S. S/ PROCESSOS
    INST.NAC.DE SEGURIDADE SOCIAL
    PASEP
    BOLSA FAMILIA
    CASA DO TRABALHADOR
    PROGRAMA CREDIBAHIA
    PROGRAMA CREAS PAEFI
    PROGRAMA PROJOVEM URBANO
    PROGRAMA CENTRO DE CAPACITACAO
    PROGRAMA DE DIREITO A CIDADANI
    PROGRAMA P JUV E F ADOLESCENTE
    PROGRAMA LIBERDADE ASSISTIDA
    PROGRAMA SENTINELA
    PROGRAMA C.ALIMENTAR LEITE/MUL
    CENTRO DE CON. D ZAZINHA CERQU
    PROG. CENTRO DE REF A MULHER
    PROGRAMA FAMILIA ACOLHEDORA
    IRRF
    E diversos nomes próprios

1) 01.404.357/0001-51 -> inválido. Utilizado para 160 nomes diversos.

2) 000.000.000-00 -> inválido

Utilizado para os credores:

    SECRETARIA DA FAZENDA-RECEITA
    CREA-BA
    ODONTO SYSTEM
    PREVIMILpp
    PENSAO ALIMENTICIA
    BANCO REAL-SMT
    I.R.R.F. - FOLHA
    EMPRESTIMO CAIXA
    I.S.S
    I.N.S.S. S/ PROCESSOS
    I.R.R.F. - PROCESSOS
    PREV.SERV.MUN.FEIRA DE SANTANA-IPFS
    CARTÃO BMG
    BIGCARD - FHFS
    MLEAL LTDA
    MAIS ODONTO
    SINDICATO SERVIDORES PUB. MUNICIPAIS
    R.S.P.P.- SMT
    PREVCASA
    PREV.SERV. MUN. FEIRA DE SANTANA
    RGPS/INSS - FOLHA
    DESC. JUDICIAL
    D. JUD. FIXO
    PREV.SERV. MUN. FEIRA DE SANTANA I.P.F.S
    I.R.R.F. - S/FOLHA
    D. FARMACIA
    SINDSAUDE
    SINDICATO S. P. MUNICIPAIS
    I.R.R.F. Retido sobre rendimentos trabal
    VIDA AZUL
    R.S.P.P.-/INSVSTASFIN SMT
    INDENIZACAO AO TESOURO
    VALE TRANSPORTES
    APLB-SIND.TRAB.EDUCACAO DO ESTADO BAHIA
    APLB-SAUDE
    ASECAMUFS
    BANCO REAL-IPFS
    BANCO  BRADESCO
    BONSUCESSO- IPFS
    DESCONTO BAHIA ODONTO
    INDENIZACAO A FAZENDA
    FARMACIA VIDA
    BIGCARD
    PREVIDONTba- IPFS
    POINT APLB- IPFS
    SINDACS-IPFS
    AASPE-IPFS
    SIND. SAUDE
    SINDASCOFE-IPFS
    AGMFS
    SIND.SAUDE
    RGPS/INSS
    MASCARENHAS & LEAL lTDA
    RGPS/INSS FC
    SINDESP - F.H.F.S.
    PREVDONTO
    I.N.S.S. S/ PROCESSOS SMT
    I.N.S.S. S/PROCESSOS
    TRIBUNAL DE JUSTICA DO E.BAHIA
    ODONTO S/A
    DESCONTO FARMACIA
    EBAL CREDFEIRA

3) 08.576.590/0001-07 ->  FUNDO MUNICIPAL DE SAUDE DE FEIRA DE SANTANA;

Também utilizado para os credores:

    FOLHA DE PGTO CEO
    FOLHA DE PAGAMENTO CEREST
    FOLHA DE PAGAMENTO SMS
    FOLHA PAGAMENTO UBS
    FOLHA PAG. FARMACIA POPULAR
    FOLHA PAGAMENTO PACS-CONTRATO
    FOLHA PAGAMENTO DST/AIDS
    FOLHA PAG. ENDEMIAS ESTATUTARI
    FOLHA PAG. ENDEMIAS-CONTRATO
    FOLHA DE PAG. VIG. EM SAUDE
    PESSOAL CIVIL
    FOLHA PAGAMENTO PACS ESTATUTAR
    FOLHA DE PAGAMENTO SAMU
    FOLHA PAG. MEDIA COMPLEXIDADE
    FOLHA DE PAGAMENTO CAPS
    I.N.S.S. S/PROCESSOS
    FOLHA DE PAGAMENTO ESTAGIARIO
    CAUCAO
    FOLHA PAG. PESSOAL CIVIL
    PREV.SERV. MUN. FEIRA DE SANTANA
    DESC. JUDICIAL(PENSÃO ALIMENTICIA
    I.S.S
    E um nome próprio.

4) 00.857.659/0001-07 -> Inválido. Utilizado para:


    FOLHA DE PAGAMENTO SMS
    FOLHA PAGAMENTO PACS ESTATUTAR
    FOLHA PAGAMENTO PACS-CONTRATO
    PESSOAL CIVIL
    FOLHA DE PAGAMENTO CAPS
    FOLHA PAG. MEDIA COMPLEXIDADE
    FOLHA DE PAGAMENTO CEREST
    FOLHA PAG. FARMACIA POPULAR
    FOLHA PAGAMENTO DST/AIDS
    FOLHA DE PAGAMENTO SAMU
    FOLHA DE PAG. VIG. EM SAUDE
    FOLHA PAG. ENDEMIAS-CONTRATO
    INSS - FOLHA
    RG FERNANDES COM. DE MAT. CIR
    I.N.S.S. S/PROCESSOS
    FOLHA DE PGTO CEO
    FOLHA PAGAMENTO UBS
    FOLHA DE PAGAMENTO ESTAGIARIO
    CAUCAO
    E 1 nome próprio. O mesmo acima:
    ALAIDE LIMA DE FREITAS. 
    Favor não divulgar o nome.



5) 07.520.940/0001-50 -> FUNDACAO MUNICIPAL DE TECNOLOGIA DA INFORMACAO, TELECOMUNICACOES E CULTURA EGBERTO TAVARES COSTA;

Também utilizado para:
    
    INST.NAC.DE SEGURIDADE SOCIAL
    PESSOAL CIVIL
    CONTRATAÇÃO TEMPORÁRIA - FCMET
    BOLSA ESTÁGIO
    CONTRATAÇÃO TEMPORÁRIA - FUNTITEC
    E outros 12 nomes próprios


6) 02.983.473/0001-81 -> SUPERINTENDENCIA MUNICIPAL DE TRANSITO;

Também utilizado para:

    EMP BMG, Bolsa Estágio, Vale Transportes, INSS,
    Empréstimo Caixa, Pensão Alimenticia, Orale, IPFS, Odonto System,
    e outros 3 nomes próprios.

7) 05.824.468/0001-14 ->  M E B SERVICOS E LOCACOES LTDA 

8) 04.063.715/0001-36 -> CNPJ inválido.

Utilizado para:

    FUNDAC. HOSPITALAR DE FSA
    PREFEITURA MUNICIPAL F.SANTANA
    OI VELOX  COBRANCA LTDA
    DIRETORIA EXEC. FUNDO NAC.DE S
    LIGCEL - F.H.F.S
    UNIAO MEDICA
    ODONTO S/A.
    A B S - F.H.F.S
    EMPRESTIMO BVF - F.H.F.S
    FINASA BMC
    B CRUZ SUL

9) 42.743.658/0001-33 ->  Dono: INSTITUTO DE PREVIDENCIA DE FEIRA DE SANTANA - IPFS;
    
    Porém também usado ORALE e um nome próprio

In [6]:
for id in ids_group[ids_group["credor"] > 9].index:
    print(id)

14.043.574/0001-51
01.404.357/0001-51
000.000.000-00
08.576.590/0001-07
00.857.659/0001-07
07.520.940/0001-50
02.983.473/0001-81
05.824.468/0001-14
04.063.715/0001-36
42.743.658/0001-33


## Despesas negativas -  _em andamento_

Valores negativos fazem sentido para despesas?

Vamos tentar ter uma ideia melhor do que sejam estes pagamentos negativos:

In [7]:
negative_payments = pagamentos[pagamentos['valor'] < 0]
print(negative_payments.info())
negative_payments

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16494 entries, 164 to 427785
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   data_publicacao          16494 non-null  object 
 1   fase                     16494 non-null  object 
 2   credor                   16494 non-null  object 
 3   valor                    16494 non-null  float64
 4   numero                   16494 non-null  object 
 5   cpf_ou_cnpj              16494 non-null  object 
 6   data_pagamento           16494 non-null  object 
 7   numero_processo          13353 non-null  object 
 8   bem_ou_servico_prestado  15793 non-null  object 
 9   natureza                 16494 non-null  object 
 10  acao                     16493 non-null  object 
 11  funcao                   16494 non-null  object 
 12  subfuncao                16494 non-null  object 
 13  processo_licitatorio     16494 non-null  object 
 14  fonte_recurso      

,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
164,04/01/2010,EMPENHO,WILLEN DA SILVA REIS,-39371.74,10000879,016.330.955-84,04/01/2010,350/2007,ANULACAO PARCIAL DO EMPENHO DE NUMERO 10000004,339039740000 - Outros Serv. de Terc.,2097 - Pavimentacao e infra-estrutura,15 - URBANISMO,451 - INFRA-ESTRUTURA URBANA,TOMADA DE PRECO,0000 - RECURSOS ORDINARIOS
219,04/01/2010,EMPENHO,COELBA,-789.55,10000287,01.513.962/0001-03,04/01/2010,NaN,ANULACAO PARCIAL DO EMPENHO DE NUMERO 10000278,339092000000 - Despesas Exercicios Anteriores,2051 - Desenvolvimento das Acoes de Saude-Admi...,10 - SAUDE,122 - ADMINISTRACAO GERAL,ISENTO,0002 - REC.IMP.TRANSF.IMP.SAUDE 15%
252,04/01/2010,EMPENHO,EMBASA-EMP BAIANA AGUA E SANE,-11.91,10000175,01.350.467/0001-10,04/01/2010,NaN,ANULACAO PARCIAL DO EMPENHO DE NUMERO 10000143,339092000000 - Despesas Exercicios Anteriores,2051 - Desenvolvimento das Acoes de Saude-Admi...,10 - SAUDE,122 - ADMINISTRACAO GERAL,ISENTO,0002 - REC.IMP.TRANSF.IMP.SAUDE 15%
545,04/01/2010,EMPENHO,LM REVENDEDORA DE G.ANDRADE LT,-150.00,10000490,00.573.999/0001-10,04/01/2010,236/2009,ANULACAO PARCIAL DO EMPENHO DE NUMERO 10000367,339030040000 - Gas Engarrafado,2037 - Manutencao desenv. da educacao infantil,12 - EDUCACAO,365 - EDUCACAO INFANTIL,PREGAO,0001 - REC.IMP.TRANSF.EDUCACAO 25%
774,04/01/2010,EMPENHO,COELBA,-325.27,10000288,01.513.962/0001-03,04/01/2010,NaN,ANULACAO PARCIAL DO EMPENHO DE NUMERO 10000279,339092000000 - Despesas Exercicios Anteriores,2051 - Desenvolvimento das Acoes de Saude-Admi...,10 - SAUDE,122 - ADMINISTRACAO GERAL,ISENTO,0002 - REC.IMP.TRANSF.IMP.SAUDE 15%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426725,06/11/2019,EMPENHO,R.C.R CONSTRUCOES ELETRICAS LTDA,-3900.00,19000045,09.598.175/0001-17,06/11/2019,262-2019-08D,ANULACAO PARCIAL DO EMPENHO DE NUMERO 19000019,339039999900000000 - Outros Serv.Terceiros - P...,idica - Demais serviços 2027 - Elabor...,04 - ADMINISTRACAO,121 - PLANEJAMENTO E ORCAMENTO,DISPENSA,0000 - RECURSOS ORDINARIOS
427144,11/11/2019,EMPENHO,TELEMAR NORTE LESTE - OI,-20000.00,19000144,33.000.118/0001-79,11/11/2019,2641/2016,ANULACAO PARCIAL DO EMPENHO DE NUMERO 19000097,339039090000000000 - Serviços de Telefonia,2004 - Manutencao do gabinete do prefeito,04 - ADMINISTRACAO,122 - ADMINISTRACAO GERAL,PREGAO,0000 - RECURSOS ORDINARIOS
427606,14/11/2019,LIQUIDAÇÃO,TC MONITORAMENTO LTDA,-6020.00,19000850/0001,11.203.800/0001-26,14/11/2019,135-2018,NaN,339039999900 - Outros Serv.Terceiros - P. Juri...,2281 - Serviço da Proteção Social Básica,08 - ASSISTENCIA SOCIAL,244 - ASSISTENCIA COMUNITARIA,PREGAO,0000 - RECURSOS ORDINARIOS
427784,20/11/2019,EMPENHO,CG TRANSPORTES URBANOS LTDA,-207776.88,19000584,04.603.903/0001-18,20/11/2019,475/2014,ANULACAO PARCIAL DO EMPENHO DE NUMERO 19000524,339039998900000000 - Outros Serv.Terceiros - P...,idica - Transporte Escolar 2039 - Oferec...,12 - EDUCACAO,368 - EDUCACAO BASICA (4),CONCORRENCIA,0001 - REC.IMP.TRANSF.EDUCACAO 25%


In [8]:
negative_payments["bem_ou_servico_prestado"] = negative_payments["bem_ou_servico_prestado"].astype(str)
set(name[:15] for name in negative_payments["bem_ou_servico_prestado"].unique())

{'02 MEDICAO DOS ',
 '02 PARCELA DA A',
 '07 E 08 PARCELA',
 '1? PARCELA DO C',
 '27 MEDICAO DOS ',
 '2ª PARCELA - PR',
 'ADIANTAMENTO EM',
 'ALUGUEL DE IMOV',
 'ANULACAO PARCIA',
 'APRESENTACAO C/',
 'AQUISICAO DE BO',
 'AQUISICAO DE CA',
 'AQUISICAO DE EQ',
 'AQUISIÇÃO DE BI',
 'AQUISIÇÃO DE CO',
 'AQUISIÇÃO DE ME',
 'AQUISIÇÃO DE PE',
 'AQUISIÇÃO DE PL',
 'COMPRA DE MATER',
 'CONFECÇÃO DE BI',
 'CONFECÇÃO DE CA',
 'CONSUMO DE TELE',
 'DESPESA COM CON',
 'EMPLACAMENTO E ',
 'FOLHA DE PAGAME',
 'FOLHA PAGAMENTO',
 'FORNEC.DE COMBU',
 'FORNEC.DE PEÇAS',
 'FORNECIMENTO DE',
 'LOCAC?O DE 15 E',
 'LOCAÇÃO DE VEIC',
 'LREF. A SERVIÇO',
 'MEDICAO UNICA D',
 'PAG. DE VALES T',
 'PAG.DE AQUISICA',
 'PAG.DE EMPLACAM',
 'PAG.DE FORNECIM',
 'PAG.DE MANUTENC',
 'PAG.DE VALE TRA',
 'PAG.DE VALES TR',
 'PAG.REF. A FONT',
 'PAG.REF. A LOCA',
 'PAG.REF.VALES T',
 'PAGT RELATIVO A',
 'PAGT. REF. AQUI',
 'PAGT. REF. CONS',
 'PAGT. REF. DIFE',
 'PAGT. REF. JURO',
 'PAGT. REF. PRES',
 'PAGT. REF. SERV',


In [9]:
print(f"Numero de pagamentos negativos: {len(negative_payments)}\n\
Este numero representa {round(len(negative_payments)*100/len(pagamentos),2)}% \
do total de pagamentos registrados")

Numero de pagamentos negativos: 16494
Este numero representa 3.85% do total de pagamentos registrados


Vamos ao [Portal da Transparência](http://www.transparencia.feiradesantana.ba.gov.br/index.php?view=despesa) conferir o que são estes valores.

O primeiro, do Banco do Brasil, constam no mesmo dia 3 pagamentos: este negativo, outro de mesmo valor e positivo, outro de valor parecido e positivo. 

O segundo, do IUNE -INST. DE. URO E NEFRO FSA, existem 10 pagamentos similares (5 de cada valor). Ao final, no balanço da conta, resta apenas um pagamento de cada valor. Parece corroborar com a hipótese anterior.

O terceiro, de SIMOES ESTRELAS EMPREENDIMENTOS LTDA ME EPP, também segue o mesmo padrão. 5 pagamentos que, somados, correspondem a 1 pagamento de um valor.

O registro 164, do dia 04/01/2010: R$ -39.371,74 de WILLEN DA SILVA REIS, CPF Y, na fase EMPENHO, registrado no número 10000879 e número do processo 350/2007. Consta no Portal: Bem / Serviço Prestado: ANULACAO PARCIAL DO EMPENHO DE NUMERO 10000004 

Parece que as despesas negativas foram registradas para balancear despesas cadastradas de forma errada



## Numero de processo - _em construção_

Existem os campos `numero` e `numero_processo`.
São dois campos diferentes.

Dos 427.873 registros de despesa,
existem 193.494 números
e 17.919 números de processo.

O que representam esses campos?


In [10]:
# Investigando numero x numero_processo. O que significa numero?
gp_numero = pagamentos.groupby('numero')
gp_numero.nunique().sort_values('numero_processo', ascending=False)

,data_publicacao,fase,credor,valor,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
numero,,,,,,,,,,,,,,
11000014,7,1,21,21,21,7,15,9,15,22,14,14,8,9
11000018,9,1,21,20,20,9,14,8,18,22,13,10,7,8
11000011,7,1,24,23,23,7,14,9,16,26,16,14,6,7
11000007,4,1,22,17,22,4,13,7,12,23,13,11,7,7
16000024,3,1,13,14,13,3,13,15,11,15,9,10,8,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14987-17,2,2,1,1,1,2,0,1,1,1,1,1,1,1
07316-17,1,2,1,1,1,1,0,1,1,1,1,1,1,1
14986-17,2,2,1,1,1,2,0,1,1,1,1,1,1,1


In [11]:
pagamentos[pagamentos['numero'] == '440/14CP'].head()

,data_publicacao,fase,credor,valor,numero,cpf_ou_cnpj,data_pagamento,numero_processo,bem_ou_servico_prestado,natureza,acao,funcao,subfuncao,processo_licitatorio,fonte_recurso
244294,04/01/2016,EMPENHO,MERCADO DE PRO.E MARK. LTDA,101082.96,440/14CP,02.620.895/0001-92,04/01/2016,440/2014,"SELEÇÃO, HABILITAÇÃO E A CONTRATAÇÃO DE 04 (QU...",339039010000000000 - Publicidades e Propaganda,2023 - Realizaçao de campanhas publicitarias,24 - COMUNICACOES,131 - COMUNICACAO SOCIAL,CONCORRENCIA,0000 - RECURSOS ORDINARIOS
244295,04/01/2016,EMPENHO,ATIVA COM.E PROPAGANDA LTDA,93265.90,440/14CP,03.696.272/0001-66,04/01/2016,440/2014,"SELEÇÃO, HABILITAÇÃO E A CONTRATAÇÃO DE 04 (QU...",339039010000000000 - Publicidades e Propaganda,2023 - Realizaçao de campanhas publicitarias,24 - COMUNICACOES,131 - COMUNICACAO SOCIAL,CONCORRENCIA,0000 - RECURSOS ORDINARIOS
245317,04/01/2016,EMPENHO,CIDADE PROP. E MARKENTING LTDA,11122.31,440/14CP,06.955.452/0001-03,04/01/2016,440/2014,"SELEÇÃO, HABILITAÇÃO E A CONTRATAÇÃO DE 04 (QU...",339039010000000000 - Publicidades e Propaganda,2023 - Realizaçao de campanhas publicitarias,24 - COMUNICACOES,131 - COMUNICACAO SOCIAL,CONCORRENCIA,0000 - RECURSOS ORDINARIOS
245318,04/01/2016,EMPENHO,A.C.PROPAGANDA LTDA.,110000.00,440/14CP,02.811.243/0001-35,04/01/2016,440/2014,"SELEÇÃO, HABILITAÇÃO E A CONTRATAÇÃO DE 04 (QU...",339039010000000000 - Publicidades e Propaganda,2023 - Realizaçao de campanhas publicitarias,24 - COMUNICACOES,131 - COMUNICACAO SOCIAL,CONCORRENCIA,0000 - RECURSOS ORDINARIOS
245319,04/01/2016,EMPENHO,MERCADO DE PRO.E MARK. LTDA,101082.96,440/14CP,02.620.895/0001-92,04/01/2016,440/2014,"SELEÇÃO, HABILITAÇÃO E A CONTRATAÇÃO DE 04 (QU...",339039010000000000 - Publicidades e Propaganda,2023 - Realizaçao de campanhas publicitarias,24 - COMUNICACOES,131 - COMUNICACAO SOCIAL,CONCORRENCIA,0000 - RECURSOS ORDINARIOS
